In [1]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

In [9]:
ws = Workspace(subscription_id="c51cd33f-083e-4304-a95b-442a52dc4a2a", 
               resource_group="NetworkWatcherRG",workspace_name="eSpaceAI", auth=None, _location="North Europe", _disable_service_check=False)

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.


You have logged in. Now let us find all the subscriptions to which you have access...


AuthenticationException: No subscriptions were found for 'None'. If this is expected, use '--allow-no-subscriptions' to have tenant level accesses

In [11]:
ws = Workspace.get(subscription_id="c51cd33f-083e-4304-a95b-442a52dc4a2a", 
               resource_group="NetworkWatcherRG",name="eSpaceAI", auth=None)

Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.


You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


In [12]:
!ls

Azure_Yolo_Frame_Diff_real_time_object_detection.py
MobileNetSSD_deploy.caffemodel
MobileNetSSD_deploy.prototxt.txt
Sample01.mp4
Traffic.mp4
Traffic2.mp4
Yolo_Frame_Diff_real_time_object_detection.py
Yolo_Tinyreal_time_object_detection.py
Yolo_real_time_object_detection.py
car1.mp4
darknet
model_deploy.ipynb
model_deploy.py
yolo


In [13]:
model = Model.register(workspace = ws,
                       model_path ="yolo/yolov3.weights",
                       model_name = "yolov3",
                       tags = {"yolov3": "yolov3"},
                       description = "Yolov3 large model",)

Registering model yolov3


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

In [18]:
salenv = CondaDependencies()
salenv.add_conda_package("compare_ssim")
salenv.add_conda_package("imutils")
salenv.add_conda_package("cv2")
 
with open("yolov3.yml","w") as f:
    f.write(salenv.serialize_to_string())
with open("yolov3.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
- compare_ssim
- imutils
- cv2



In [17]:
%%writefile score.py
 
import json
import numpy as np
import os
import cv2
 
from azureml.core.model import Model
 
def init(src):
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('yolov3')
    model = joblib.load(model_path)
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Writing score.py
